# Data Cleaning

In [1]:
import pandas as pd
import scipy.stats
import numpy as np

## Read & Inspect Data

In [2]:
card = pd.read_excel('card transactions.xlsx')

In [3]:
card.head(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
5,6,5142149874,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0
6,7,5142189277,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0
9,10,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [4]:
card.shape

(96753, 10)

In [5]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96753 entries, 0 to 96752
Data columns (total 10 columns):
Recnum               96753 non-null int64
Cardnum              96753 non-null int64
Date                 96753 non-null datetime64[ns]
Merchnum             93378 non-null object
Merch description    96753 non-null object
Merch state          95558 non-null object
Merch zip            92097 non-null float64
Transtype            96753 non-null object
Amount               96753 non-null float64
Fraud                96753 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 7.4+ MB


## Remove Outliers

In [6]:
card = card[card.Transtype == 'P']
card = card[card.Recnum != 52715]

## Fill Missing Values

In [7]:
## fill merch state: group by merch zip or group by all
state_ref = card.groupby('Merch zip')['Merch state'].apply(lambda x: x.mode()).reset_index()

state_ref = state_ref[state_ref.level_1 == 0].iloc[:, [0,2]]

In [8]:
state_ref.columns = ['Merch zip', 'state ref']

In [9]:
card = card.merge(state_ref, on = 'Merch zip', how='left')

In [10]:
card['Merch state'] = card['Merch state'].fillna(card['state ref']).fillna('TN')

In [11]:
card['Merch state'].isnull().sum()

0

In [12]:
## fill merch zip: group by cardnum and merch state, or merch state, or all
zip_ref = card.groupby(['Merch state', 'Cardnum'])['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref = zip_ref[zip_ref.level_2 == 0].iloc[:, [0,1,3]]

In [13]:
zip_ref.columns = ['Merch state', 'Cardnum', 'zip ref']

In [14]:
card = card.merge(zip_ref, on = ['Merch state', 'Cardnum'], how='left')

In [15]:
zip_ref2 = card.groupby(['Merch state'])['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref2 = zip_ref2[zip_ref2.level_1 == 0].iloc[:, [0,2]]
zip_ref2.columns = ['Merch state', 'zip ref2']

In [16]:
card = card.merge(zip_ref2, on = 'Merch state', how='left')

In [17]:
card['Merch zip'] = card['Merch zip'].fillna(card['zip ref']).fillna(card['zip ref2']).fillna(38118.0)

In [18]:
card['Merch zip'].isnull().sum()

0

In [19]:
## fill merchnum: group by cardnum, merchstate, or merchstate, or all
merch_ref = card.groupby(['Merch state', 'Cardnum'])['Merchnum'].apply(lambda x: x.mode()).reset_index()
merch_ref = merch_ref[merch_ref.level_2 == 0].iloc[:, [0,1,3]]

In [20]:
merch_ref.columns = ['Merch state', 'Cardnum', 'merch ref']

In [21]:
card = card.merge(merch_ref, on = ['Merch state', 'Cardnum'], how='left')

In [22]:
merch_ref2 = card.groupby(['Merch state'])['Merchnum'].apply(lambda x: x.mode()).reset_index()
merch_ref2 = merch_ref2[merch_ref2.level_1 == 0].iloc[:, [0,2]]
merch_ref2.columns = ['Merch state', 'merch ref2']

In [23]:
card = card.merge(merch_ref2, on = 'Merch state', how='left')

In [24]:
card['Merchnum'] = card['Merchnum'].fillna(card['merch ref']).fillna(card['merch ref2']).fillna('930090121224')

In [25]:
card['Merchnum'].isnull().sum()

0

In [26]:
card.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96396
Data columns (total 15 columns):
Recnum               96397 non-null int64
Cardnum              96397 non-null int64
Date                 96397 non-null datetime64[ns]
Merchnum             96397 non-null object
Merch description    96397 non-null object
Merch state          96397 non-null object
Merch zip            96397 non-null float64
Transtype            96397 non-null object
Amount               96397 non-null float64
Fraud                96397 non-null int64
state ref            92030 non-null object
zip ref              95195 non-null float64
zip ref2             96205 non-null float64
merch ref            95333 non-null object
merch ref2           96379 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 11.8+ MB


In [27]:
card_clean = card.loc[:, 'Recnum':'Fraud']

In [28]:
card_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96396
Data columns (total 10 columns):
Recnum               96397 non-null int64
Cardnum              96397 non-null int64
Date                 96397 non-null datetime64[ns]
Merchnum             96397 non-null object
Merch description    96397 non-null object
Merch state          96397 non-null object
Merch zip            96397 non-null float64
Transtype            96397 non-null object
Amount               96397 non-null float64
Fraud                96397 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 8.1+ MB


# Feature Engineer

## Cardnum Amount Expert Variables

In [29]:
card_clean.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [30]:
card_clean['order'] = card_clean.groupby(['Cardnum', 'Date']).cumcount() + 1

In [31]:
card_clean['order_merch'] = card_clean.groupby(['Merchnum', 'Date']).cumcount() + 1

In [32]:
card_clean['order_cm'] = card_clean.groupby(['Cardnum', 'Merchnum', 'Date']).cumcount() + 1

In [33]:
card_clean['order_cz'] = card_clean.groupby(['Cardnum', 'Merch zip', 'Date']).cumcount() + 1

In [34]:
card_clean['order_cs'] = card_clean.groupby(['Cardnum', 'Merch state', 'Date']).cumcount() + 1

In [35]:
card_clean.head(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,order,order_merch,order_cm,order_cz,order_cs
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,1,1,1,1,1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,1,1,1,1,1
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,1,1,1,1,1
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,1,2,1,1,1
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,2,3,2,2,2
5,6,5142149874,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0,1,4,1,1,1
6,7,5142189277,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,1,5,1,1,1
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0,1,1,1,1,1
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0,1,1,1,1,1
9,10,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,3,6,3,3,3


In [36]:
card_ref = card_clean.copy()

In [37]:
card_ref = card_ref.set_index('Date')

In [38]:
card_ref.groupby(['Cardnum'])['Amount'].rolling('1d').mean().reset_index().head(20)

,Cardnum,Date,Amount
0,5142110002,2010-10-12,150.0000
1,5142110081,2010-03-08,495.9000
2,5142110081,2010-03-08,566.0500
3,5142110081,2010-11-26,636.2000
4,5142110081,2010-12-27,495.9000
5,5142110313,2010-10-07,144.0000
6,5142110313,2010-10-07,94.0000
7,5142110313,2010-10-07,96.0000
8,5142110402,2010-01-19,506.5000
9,5142110402,2010-01-31,20.8100


In [39]:
card_ref[card_ref.Cardnum == 5142110081]

,Recnum,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,order,order_merch,order_cm,order_cz,order_cs
Date,,,,,,,,,,,,,,
2010-03-08,16628,5142110081,930090121224,X OFFICE DEPOT #1,TN,38118.0,P,495.9,0,1,5,1,1,1
2010-03-08,16801,5142110081,930090121224,X FRANKLIN COVEY,TN,38118.0,P,636.2,0,2,20,2,2,2
2010-11-26,89482,5142110081,2094330000009,FRANKLIN COVEY COMPANY,UT,84119.0,P,636.2,0,1,2,1,1,1
2010-12-27,95973,5142110081,4503082412500,OFFICE DEPOT #1078,WA,98032.0,P,495.9,0,1,1,1,1,1


In [40]:
%%time
## avg

avg_card_1 = card_ref.groupby(['Cardnum'])['Amount'].rolling('1d').mean().reset_index()
avg_card_1['order'] = avg_card_1.groupby(['Cardnum', 'Date']).cumcount() + 1

avg_card_7 = card_ref.groupby(['Cardnum'])['Amount'].rolling('7d').mean().reset_index()
avg_card_7['order'] = avg_card_7.groupby(['Cardnum', 'Date']).cumcount() + 1

avg_card_14 = card_ref.groupby(['Cardnum'])['Amount'].rolling('14d').mean().reset_index()
avg_card_14['order'] = avg_card_14.groupby(['Cardnum', 'Date']).cumcount() + 1

avg_card_30 = card_ref.groupby(['Cardnum'])['Amount'].rolling('30d').mean().reset_index()
avg_card_30['order'] = avg_card_30.groupby(['Cardnum', 'Date']).cumcount() + 1

CPU times: user 3.47 s, sys: 46.6 ms, total: 3.51 s
Wall time: 3.54 s


In [41]:
%%time
## max

max_card_1 = card_ref.groupby(['Cardnum'])['Amount'].rolling('1d').max().reset_index()
max_card_1['order'] = max_card_1.groupby(['Cardnum', 'Date']).cumcount() + 1

max_card_7 = card_ref.groupby(['Cardnum'])['Amount'].rolling('7d').max().reset_index()
max_card_7['order'] = max_card_7.groupby(['Cardnum', 'Date']).cumcount() + 1

max_card_14 = card_ref.groupby(['Cardnum'])['Amount'].rolling('14d').max().reset_index()
max_card_14['order'] = max_card_14.groupby(['Cardnum', 'Date']).cumcount() + 1

max_card_30 = card_ref.groupby(['Cardnum'])['Amount'].rolling('30d').max().reset_index()
max_card_30['order'] = max_card_30.groupby(['Cardnum', 'Date']).cumcount() + 1

CPU times: user 3.76 s, sys: 41.8 ms, total: 3.8 s
Wall time: 3.83 s


In [42]:
%%time
## median

med_card_1 = card_ref.groupby(['Cardnum'])['Amount'].rolling('1d').median().reset_index()
med_card_1['order'] = med_card_1.groupby(['Cardnum', 'Date']).cumcount() + 1

med_card_7 = card_ref.groupby(['Cardnum'])['Amount'].rolling('7d').median().reset_index()
med_card_7['order'] = med_card_7.groupby(['Cardnum', 'Date']).cumcount() + 1

med_card_14 = card_ref.groupby(['Cardnum'])['Amount'].rolling('14d').median().reset_index()
med_card_14['order'] = med_card_14.groupby(['Cardnum', 'Date']).cumcount() + 1

med_card_30 = card_ref.groupby(['Cardnum'])['Amount'].rolling('30d').median().reset_index()
med_card_30['order'] = med_card_30.groupby(['Cardnum', 'Date']).cumcount() + 1

CPU times: user 3.73 s, sys: 43 ms, total: 3.78 s
Wall time: 3.8 s


In [43]:
%%time
## total

tot_card_1 = card_ref.groupby(['Cardnum'])['Amount'].rolling('1d').sum().reset_index()
tot_card_1['order'] = tot_card_1.groupby(['Cardnum', 'Date']).cumcount() + 1

tot_card_7 = card_ref.groupby(['Cardnum'])['Amount'].rolling('7d').sum().reset_index()
tot_card_7['order'] = tot_card_7.groupby(['Cardnum', 'Date']).cumcount() + 1

tot_card_14 = card_ref.groupby(['Cardnum'])['Amount'].rolling('14d').sum().reset_index()
tot_card_14['order'] = tot_card_14.groupby(['Cardnum', 'Date']).cumcount() + 1

tot_card_30 = card_ref.groupby(['Cardnum'])['Amount'].rolling('30d').sum().reset_index()
tot_card_30['order'] = tot_card_30.groupby(['Cardnum', 'Date']).cumcount() + 1

CPU times: user 3.49 s, sys: 40.2 ms, total: 3.53 s
Wall time: 3.55 s


In [44]:
card_clean.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,order,order_merch,order_cm,order_cz,order_cs
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,1,1,1,1,1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,1,1,1,1,1
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,1,1,1,1,1
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,1,2,1,1,1
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,2,3,2,2,2


In [45]:
## compare with actual

amount_card = card_clean.iloc[:, [0,1,2,8,10]].merge(avg_card_1, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(avg_card_7, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(avg_card_14, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(avg_card_30, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(tot_card_1, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(tot_card_7, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(tot_card_14, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(tot_card_30, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(max_card_1, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(max_card_7, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(max_card_14, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(max_card_30, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(med_card_1, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(med_card_7, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(med_card_14, on = ['Cardnum', 'Date', 'order'], how = 'inner').merge(med_card_30, on = ['Cardnum', 'Date', 'order'], how = 'inner')



In [46]:
amount_card.head()

,Recnum,Cardnum,Date,Amount_x,order,Amount_y,Amount_x,Amount_y,Amount_x,Amount_y,...,Amount_y,Amount_x,Amount_y,Amount_x,Amount_y,Amount_x,Amount_y,Amount_x,Amount_y,Amount
0,1,5142190439,2010-01-01,3.62,1,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
1,2,5142183973,2010-01-01,31.42,1,31.42,31.42,31.42,31.42,31.42,...,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42
2,3,5142131721,2010-01-01,178.49,1,178.49,178.49,178.49,178.49,178.49,...,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49
3,4,5142148452,2010-01-01,3.62,1,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
4,5,5142190439,2010-01-01,3.62,2,3.62,3.62,3.62,3.62,7.24,...,7.24,7.24,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62


In [47]:
amount_card.columns = ['Recnum','Cardnum', 'Date', 'Amount', 'order', 'avg_card_1', 'avg_card_7', 'avg_card_14',
       'avg_card_30', 'tot_card_1', 'tot_card_7', 'tot_card_14', 'tot_card_30',
                      'max_card_1', 'max_card_7', 'max_card_14', 'max_card_30',
                      'med_card_1', 'med_card_7', 'med_card_14', 'med_card_30']

In [48]:
### actual - average

amount_card['daa_card_1'] = amount_card['Amount'] - amount_card['avg_card_1']

amount_card['daa_card_7'] = amount_card['Amount'] - amount_card['avg_card_7']

amount_card['daa_card_14'] = amount_card['Amount'] - amount_card['avg_card_14']

amount_card['daa_card_30'] = amount_card['Amount'] - amount_card['avg_card_30']

In [49]:
### actual - median

amount_card['dam_card_1'] = amount_card['Amount'] - amount_card['med_card_1']

amount_card['dam_card_7'] = amount_card['Amount'] - amount_card['med_card_7']

amount_card['dam_card_14'] = amount_card['Amount'] - amount_card['med_card_14']

amount_card['dam_card_30'] = amount_card['Amount'] - amount_card['med_card_30']



In [50]:
### actual / average

amount_card['qaa_card_1'] = amount_card['Amount'] / amount_card['avg_card_1']

amount_card['qaa_card_7'] = amount_card['Amount'] / amount_card['avg_card_7']

amount_card['qaa_card_14'] = amount_card['Amount'] / amount_card['avg_card_14']

amount_card['qaa_card_30'] = amount_card['Amount'] / amount_card['avg_card_30']


In [51]:
### actual / total

amount_card['qat_card_1'] = amount_card['Amount'] / amount_card['tot_card_1']

amount_card['qat_card_7'] = amount_card['Amount'] / amount_card['tot_card_7']

amount_card['qat_card_14'] = amount_card['Amount'] / amount_card['tot_card_14']

amount_card['qat_card_30'] = amount_card['Amount'] / amount_card['tot_card_30']



In [52]:
### actual / max

amount_card['qam_card_1'] = amount_card['Amount'] / amount_card['max_card_1']

amount_card['qam_card_7'] = amount_card['Amount'] / amount_card['max_card_7']

amount_card['qam_card_14'] = amount_card['Amount'] / amount_card['max_card_14']

amount_card['qam_card_30'] = amount_card['Amount'] / amount_card['max_card_30']


In [53]:
### actual / median

amount_card['qamed_card_1'] = amount_card['Amount'] / amount_card['med_card_1']

amount_card['qamed_card_7'] = amount_card['Amount'] / amount_card['med_card_7']

amount_card['qamed_card_14'] = amount_card['Amount'] / amount_card['med_card_14']

amount_card['qamed_card_30'] = amount_card['Amount'] / amount_card['med_card_30']



## Merchant Amount Expert Variables

In [54]:
%%time
## avg

avg_merch_1 = card_ref.groupby(['Merchnum'])['Amount'].rolling('1d').mean().reset_index()
avg_merch_1['order_merch'] = avg_merch_1.groupby(['Merchnum', 'Date']).cumcount() + 1

avg_merch_7 = card_ref.groupby(['Merchnum'])['Amount'].rolling('7d').mean().reset_index()
avg_merch_7['order_merch'] = avg_merch_7.groupby(['Merchnum', 'Date']).cumcount() + 1

avg_merch_14 = card_ref.groupby(['Merchnum'])['Amount'].rolling('14d').mean().reset_index()
avg_merch_14['order_merch'] = avg_merch_14.groupby(['Merchnum', 'Date']).cumcount() + 1

avg_merch_30 = card_ref.groupby(['Merchnum'])['Amount'].rolling('30d').mean().reset_index()
avg_merch_30['order_merch'] = avg_merch_30.groupby(['Merchnum', 'Date']).cumcount() + 1

CPU times: user 26.2 s, sys: 284 ms, total: 26.5 s
Wall time: 26.6 s


In [55]:
%%time
## total

tot_merch_1 = card_ref.groupby(['Merchnum'])['Amount'].rolling('1d').sum().reset_index()
tot_merch_1['order_merch'] = tot_merch_1.groupby(['Merchnum', 'Date']).cumcount() + 1

tot_merch_7 = card_ref.groupby(['Merchnum'])['Amount'].rolling('7d').sum().reset_index()
tot_merch_7['order_merch'] = tot_merch_7.groupby(['Merchnum', 'Date']).cumcount() + 1

tot_merch_14 = card_ref.groupby(['Merchnum'])['Amount'].rolling('14d').sum().reset_index()
tot_merch_14['order_merch'] = tot_merch_14.groupby(['Merchnum', 'Date']).cumcount() + 1

tot_merch_30 = card_ref.groupby(['Merchnum'])['Amount'].rolling('30d').sum().reset_index()
tot_merch_30['order_merch'] = tot_merch_30.groupby(['Merchnum', 'Date']).cumcount() + 1

CPU times: user 26.6 s, sys: 274 ms, total: 26.9 s
Wall time: 27 s


In [56]:
%%time
## median

med_merch_1 = card_ref.groupby(['Merchnum'])['Amount'].rolling('1d').median().reset_index()
med_merch_1['order_merch'] = med_merch_1.groupby(['Merchnum', 'Date']).cumcount() + 1

med_merch_7 = card_ref.groupby(['Merchnum'])['Amount'].rolling('7d').median().reset_index()
med_merch_7['order_merch'] = med_merch_7.groupby(['Merchnum', 'Date']).cumcount() + 1

med_merch_14 = card_ref.groupby(['Merchnum'])['Amount'].rolling('14d').median().reset_index()
med_merch_14['order_merch'] = med_merch_14.groupby(['Merchnum', 'Date']).cumcount() + 1

med_merch_30 = card_ref.groupby(['Merchnum'])['Amount'].rolling('30d').median().reset_index()
med_merch_30['order_merch'] = med_merch_30.groupby(['Merchnum', 'Date']).cumcount() + 1

CPU times: user 26 s, sys: 192 ms, total: 26.2 s
Wall time: 26.3 s


In [57]:
%%time
## max

max_merch_1 = card_ref.groupby(['Merchnum'])['Amount'].rolling('1d').max().reset_index()
max_merch_1['order_merch'] = max_merch_1.groupby(['Merchnum', 'Date']).cumcount() + 1

max_merch_7 = card_ref.groupby(['Merchnum'])['Amount'].rolling('7d').max().reset_index()
max_merch_7['order_merch'] = max_merch_7.groupby(['Merchnum', 'Date']).cumcount() + 1

max_merch_14 = card_ref.groupby(['Merchnum'])['Amount'].rolling('14d').max().reset_index()
max_merch_14['order_merch'] = max_merch_14.groupby(['Merchnum', 'Date']).cumcount() + 1

max_merch_30 = card_ref.groupby(['Merchnum'])['Amount'].rolling('30d').max().reset_index()
max_merch_30['order_merch'] = max_merch_30.groupby(['Merchnum', 'Date']).cumcount() + 1

CPU times: user 26.9 s, sys: 212 ms, total: 27.1 s
Wall time: 27.2 s


In [58]:
card_clean.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,order,order_merch,order_cm,order_cz,order_cs
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,1,1,1,1,1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,1,1,1,1,1
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,1,1,1,1,1
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,1,2,1,1,1
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,2,3,2,2,2


In [59]:
## compare with actual

amount_merch = card_clean.iloc[:, [0,2,3,8,11]].merge(avg_merch_1, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(avg_merch_7, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(avg_merch_14, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(avg_merch_30, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(tot_merch_1, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(tot_merch_7, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(tot_merch_14, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(tot_merch_30, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(med_merch_1, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(med_merch_7, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(med_merch_14, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(med_merch_30, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(max_merch_1, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(max_merch_7, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(max_merch_14, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner').merge(max_merch_30, on = ['Merchnum', 'Date', 'order_merch'], how = 'inner')


amount_merch.columns = ['Recnum', 'Date', 'Merchnum', 'Amount', 'order_merch', 'avg_merch_1', 'avg_merch_7', 'avg_merch_14',
       'avg_merch_30', 'tot_merch_1', 'tot_merch_7', 'tot_merch_14', 'tot_merch_30',
                       'med_merch_1', 'med_merch_7', 'med_merch_14', 'med_merch_30',
                       'max_merch_1', 'max_merch_7', 'max_merch_14', 'max_merch_30']


In [60]:
### actual - average

amount_merch['daa_merch_1'] = amount_merch['Amount'] - amount_merch['avg_merch_1']

amount_merch['daa_merch_7'] = amount_merch['Amount'] - amount_merch['avg_merch_7']

amount_merch['daa_merch_14'] = amount_merch['Amount'] - amount_merch['avg_merch_14']

amount_merch['daa_merch_30'] = amount_merch['Amount'] - amount_merch['avg_merch_30']

In [61]:
### actual - median


amount_merch['dam_merch_1'] = amount_merch['Amount'] - amount_merch['med_merch_1']

amount_merch['dam_merch_7'] = amount_merch['Amount'] - amount_merch['med_merch_7']

amount_merch['dam_merch_14'] = amount_merch['Amount'] - amount_merch['med_merch_14']

amount_merch['dam_merch_30'] = amount_merch['Amount'] - amount_merch['med_merch_30']



In [62]:
### actual / average

amount_merch['qaa_merch_1'] = amount_merch['Amount'] / amount_merch['avg_merch_1']

amount_merch['qaa_merch_7'] = amount_merch['Amount'] / amount_merch['avg_merch_7']

amount_merch['qaa_merch_14'] = amount_merch['Amount'] / amount_merch['avg_merch_14']

amount_merch['qaa_merch_30'] = amount_merch['Amount'] / amount_merch['avg_merch_30']

### actual / total

amount_merch['qat_merch_1'] = amount_merch['Amount'] / amount_merch['tot_merch_1']

amount_merch['qat_merch_7'] = amount_merch['Amount'] / amount_merch['tot_merch_7']

amount_merch['qat_merch_14'] = amount_merch['Amount'] / amount_merch['tot_merch_14']

amount_merch['qat_merch_30'] = amount_merch['Amount'] / amount_merch['tot_merch_30']


In [63]:
### actual / max

amount_merch['qam_merch_1'] = amount_merch['Amount'] / amount_merch['max_merch_1']

amount_merch['qam_merch_7'] = amount_merch['Amount'] / amount_merch['max_merch_7']

amount_merch['qam_merch_14'] = amount_merch['Amount'] / amount_merch['max_merch_14']

amount_merch['qam_merch_30'] = amount_merch['Amount'] / amount_merch['max_merch_30']

In [64]:
### actual / median

amount_merch['qamed_merch_1'] = amount_merch['Amount'] / amount_merch['med_merch_1']

amount_merch['qamed_merch_7'] = amount_merch['Amount'] / amount_merch['med_merch_7']

amount_merch['qamed_merch_14'] = amount_merch['Amount'] / amount_merch['med_merch_14']

amount_merch['qamed_merch_30'] = amount_merch['Amount'] / amount_merch['med_merch_30']

## Cardnum + Merch Amount Expert Variables

In [65]:
%%time
avg_cm_1 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('1d').mean().reset_index()
avg_cm_1['order_cm'] = avg_cm_1.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

avg_cm_7 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('7d').mean().reset_index()
avg_cm_7['order_cm'] = avg_cm_7.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

avg_cm_14 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('14d').mean().reset_index()
avg_cm_14['order_cm'] = avg_cm_14.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

avg_cm_30 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('30d').mean().reset_index()
avg_cm_30['order_cm'] = avg_cm_30.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

tot_cm_1 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('1d').sum().reset_index()
tot_cm_1['order_cm'] = tot_cm_1.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

tot_cm_7 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('7d').sum().reset_index()
tot_cm_7['order_cm'] = tot_cm_7.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

tot_cm_14 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('14d').sum().reset_index()
tot_cm_14['order_cm'] = tot_cm_14.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

tot_cm_30 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('30d').sum().reset_index()
tot_cm_30['order_cm'] = tot_cm_30.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

max_cm_1 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('1d').max().reset_index()
max_cm_1['order_cm'] = max_cm_1.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

max_cm_7 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('7d').max().reset_index()
max_cm_7['order_cm'] = max_cm_7.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

max_cm_14 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('14d').max().reset_index()
max_cm_14['order_cm'] = max_cm_14.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

max_cm_30 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('30d').max().reset_index()
max_cm_30['order_cm'] = max_cm_30.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

med_cm_1 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('1d').median().reset_index()
med_cm_1['order_cm'] = med_cm_1.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

med_cm_7 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('7d').median().reset_index()
med_cm_7['order_cm'] = med_cm_7.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

med_cm_14 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('14d').median().reset_index()
med_cm_14['order_cm'] = med_cm_14.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

med_cm_30 = card_ref.groupby(['Cardnum', 'Merchnum'])['Amount'].rolling('30d').median().reset_index()
med_cm_30['order_cm'] = med_cm_30.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1


CPU times: user 5min 20s, sys: 3.77 s, total: 5min 24s
Wall time: 5min 26s


In [66]:
## merge tables

amount_cm = card_clean.iloc[:, [0,1,2,3,8,12]].merge(avg_cm_1, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(avg_cm_7, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(avg_cm_14, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(avg_cm_30, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(tot_cm_1, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(tot_cm_7, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(tot_cm_14, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(tot_cm_30, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(max_cm_1, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(max_cm_7, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(max_cm_14, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(max_cm_30, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(med_cm_1, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(med_cm_7, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(med_cm_14, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner').merge(med_cm_30, on = ['Cardnum','Merchnum', 'Date','order_cm'], how = 'inner')



amount_cm.columns = ['Recnum','Cardnum', 'Date', 'Merchnum', 'Amount','order_cm', 'avg_cm_1', 'avg_cm_7', 'avg_cm_14',
                     'avg_cm_30', 'tot_cm_1', 'tot_cm_7', 'tot_cm_14', 'tot_cm_30', 'max_cm_1', 'max_cm_7', 'max_cm_14', 'max_cm_30',
                     'med_cm_1','med_cm_7','med_cm_14','med_cm_30' ]

In [67]:
## actual - average

amount_cm['daa_cm_1'] = amount_cm['Amount'] - amount_cm['avg_cm_1']

amount_cm['daa_cm_7'] = amount_cm['Amount'] - amount_cm['avg_cm_7']

amount_cm['daa_cm_14'] = amount_cm['Amount'] - amount_cm['avg_cm_14']

amount_cm['daa_cm_30'] = amount_cm['Amount'] - amount_cm['avg_cm_30']


## actual - median

amount_cm['dam_cm_1'] = amount_cm['Amount'] - amount_cm['med_cm_1']

amount_cm['dam_cm_7'] = amount_cm['Amount'] - amount_cm['med_cm_7']

amount_cm['dam_cm_14'] = amount_cm['Amount'] - amount_cm['med_cm_14']

amount_cm['dam_cm_30'] = amount_cm['Amount'] - amount_cm['med_cm_30']


## actual / average

amount_cm['qaa_cm_1'] = amount_cm['Amount'] / amount_cm['avg_cm_1']

amount_cm['qaa_cm_7'] = amount_cm['Amount'] / amount_cm['avg_cm_7']

amount_cm['qaa_cm_14'] = amount_cm['Amount'] / amount_cm['avg_cm_14']

amount_cm['qaa_cm_30'] = amount_cm['Amount'] / amount_cm['avg_cm_30']



## actual / max

amount_cm['qam_cm_1'] = amount_cm['Amount'] / amount_cm['max_cm_1']

amount_cm['qam_cm_7'] = amount_cm['Amount'] / amount_cm['max_cm_7']

amount_cm['qam_cm_14'] = amount_cm['Amount'] / amount_cm['max_cm_14']

amount_cm['qam_cm_30'] = amount_cm['Amount'] / amount_cm['max_cm_30']



## actual / total

amount_cm['qat_cm_1'] = amount_cm['Amount'] / amount_cm['tot_cm_1']

amount_cm['qat_cm_7'] = amount_cm['Amount'] / amount_cm['tot_cm_7']

amount_cm['qat_cm_14'] = amount_cm['Amount'] / amount_cm['tot_cm_14']

amount_cm['qat_cm_30'] = amount_cm['Amount'] / amount_cm['tot_cm_30']



## actual / median

amount_cm['qamed_cm_1'] = amount_cm['Amount'] / amount_cm['med_cm_1']

amount_cm['qamed_cm_7'] = amount_cm['Amount'] / amount_cm['med_cm_7']

amount_cm['qamed_cm_14'] = amount_cm['Amount'] / amount_cm['med_cm_14']

amount_cm['qamed_cm_30'] = amount_cm['Amount'] / amount_cm['med_cm_30']




## Cardnum + Zip Amount Expert Variables

In [68]:
card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('7d').sum().head(20)

Cardnum     Merch zip  Date      
5142110002  22202.0    2010-10-12     150.00
5142110081  38118.0    2010-03-08     495.90
                       2010-03-08    1132.10
            84119.0    2010-11-26     636.20
            98032.0    2010-12-27     495.90
5142110313  38118.0    2010-10-07     144.00
                       2010-10-07     188.00
                       2010-10-07     288.00
5142110402  21090.0    2010-01-31      20.81
                       2010-02-04     458.57
            21202.0    2010-02-03      30.26
            38118.0    2010-03-08     101.50
                       2010-03-08     131.76
                       2010-03-08     569.52
                       2010-03-08     590.33
                       2010-03-08    1096.83
                       2010-03-08    1150.83
            60007.0    2010-01-19     506.50
            80249.0    2010-02-04      54.00
5142110434  46032.0    2010-10-06    1551.64
Name: Amount, dtype: float64

In [69]:
test = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('7d').sum().reset_index()

In [70]:
test['order_cz'] = test.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

In [71]:
test.head(20)

,Cardnum,Merch zip,Date,Amount,order_cz
0,5142110002,22202.0,2010-10-12,150.00,1
1,5142110081,38118.0,2010-03-08,495.90,1
2,5142110081,38118.0,2010-03-08,1132.10,2
3,5142110081,84119.0,2010-11-26,636.20,1
4,5142110081,98032.0,2010-12-27,495.90,1
5,5142110313,38118.0,2010-10-07,144.00,1
6,5142110313,38118.0,2010-10-07,188.00,2
7,5142110313,38118.0,2010-10-07,288.00,3
8,5142110402,21090.0,2010-01-31,20.81,1
9,5142110402,21090.0,2010-02-04,458.57,1


In [72]:
%time
avg_cz_1 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('1d').mean().reset_index()
avg_cz_1['order_cz'] = avg_cz_1.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

avg_cz_7 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('7d').mean().reset_index()
avg_cz_7['order_cz'] = avg_cz_7.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

avg_cz_14 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('14d').mean().reset_index()
avg_cz_14['order_cz'] = avg_cz_14.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

avg_cz_30 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('30d').mean().reset_index()
avg_cz_30['order_cz'] = avg_cz_30.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

tot_cz_1 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('1d').sum().reset_index()
tot_cz_1['order_cz'] = tot_cz_1.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

tot_cz_7 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('7d').sum().reset_index()
tot_cz_7['order_cz'] = tot_cz_7.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

tot_cz_14 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('14d').sum().reset_index()
tot_cz_14['order_cz'] = tot_cz_14.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

tot_cz_30 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('30d').sum().reset_index()
tot_cz_30['order_cz'] = tot_cz_30.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

med_cz_1 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('1d').median().reset_index()
med_cz_1['order_cz'] = med_cz_1.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

med_cz_7 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('7d').median().reset_index()
med_cz_7['order_cz'] = med_cz_7.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

med_cz_14 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('14d').median().reset_index()
med_cz_14['order_cz'] = med_cz_14.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

med_cz_30 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('30d').median().reset_index()
med_cz_30['order_cz'] = med_cz_30.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

max_cz_1 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('1d').max().reset_index()
max_cz_1['order_cz'] = max_cz_1.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

max_cz_7 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('7d').max().reset_index()
max_cz_7['order_cz'] = max_cz_7.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

max_cz_14 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('14d').max().reset_index()
max_cz_14['order_cz'] = max_cz_14.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

max_cz_30 = card_ref.groupby(['Cardnum', 'Merch zip'])['Amount'].rolling('30d').max().reset_index()
max_cz_30['order_cz'] = max_cz_30.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1




CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [73]:
amount_cz = card_clean.iloc[:, [0,1,2,6,8,13]].merge(avg_cz_1, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(avg_cz_7, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(avg_cz_14, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(avg_cz_30, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(tot_cz_1, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(tot_cz_7, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(tot_cz_14, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(tot_cz_30, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(max_cz_1, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(max_cz_7, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(max_cz_14, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(max_cz_30, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(med_cz_1, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(med_cz_7, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(med_cz_14, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner').merge(med_cz_30, on = ['Cardnum','Merch zip', 'Date','order_cz'], how = 'inner')


amount_cz.columns = ['Recnum','Cardnum', 'Date', 'Merch zip', 'Amount','order_cz', 'avg_cz_1', 'avg_cz_7', 'avg_cz_14',
                     'avg_cz_30', 'tot_cz_1', 'tot_cz_7', 'tot_cz_14', 'tot_cz_30', 'max_cz_1', 'max_cz_7', 'max_cz_14', 'max_cz_30',
                     'med_cz_1','med_cz_7','med_cz_14','med_cz_30' ]


In [74]:
## actual - average

amount_cz['daa_cz_1'] = amount_cz['Amount'] - amount_cz['avg_cz_1']

amount_cz['daa_cz_7'] = amount_cz['Amount'] - amount_cz['avg_cz_7']

amount_cz['daa_cz_14'] = amount_cz['Amount'] - amount_cz['avg_cz_14']

amount_cz['daa_cz_30'] = amount_cz['Amount'] - amount_cz['avg_cz_30']


## actual - median

amount_cz['dam_cz_1'] = amount_cz['Amount'] - amount_cz['med_cz_1']

amount_cz['dam_cz_7'] = amount_cz['Amount'] - amount_cz['med_cz_7']

amount_cz['dam_cz_14'] = amount_cz['Amount'] - amount_cz['med_cz_14']

amount_cz['dam_cz_30'] = amount_cz['Amount'] - amount_cz['med_cz_30']


## actual / average

amount_cz['qaa_cz_1'] = amount_cz['Amount'] / amount_cz['avg_cz_1']

amount_cz['qaa_cz_7'] = amount_cz['Amount'] / amount_cz['avg_cz_7']

amount_cz['qaa_cz_14'] = amount_cz['Amount'] / amount_cz['avg_cz_14']

amount_cz['qaa_cz_30'] = amount_cz['Amount'] / amount_cz['avg_cz_30']



## actual / max

amount_cz['qam_cz_1'] = amount_cz['Amount'] / amount_cz['max_cz_1']

amount_cz['qam_cz_7'] = amount_cz['Amount'] / amount_cz['max_cz_7']

amount_cz['qam_cz_14'] = amount_cz['Amount'] / amount_cz['max_cz_14']

amount_cz['qam_cz_30'] = amount_cz['Amount'] / amount_cz['max_cz_30']



## actual / total

amount_cz['qat_cz_1'] = amount_cz['Amount'] / amount_cz['tot_cz_1']

amount_cz['qat_cz_7'] = amount_cz['Amount'] / amount_cz['tot_cz_7']

amount_cz['qat_cz_14'] = amount_cz['Amount'] / amount_cz['tot_cz_14']

amount_cz['qat_cz_30'] = amount_cz['Amount'] / amount_cz['tot_cz_30']


## actual / median

amount_cz['qamed_cz_1'] = amount_cz['Amount'] / amount_cz['med_cz_1']

amount_cz['qamed_cz_7'] = amount_cz['Amount'] / amount_cz['med_cz_7']

amount_cz['qamed_cz_14'] = amount_cz['Amount'] / amount_cz['med_cz_14']

amount_cz['qamed_cz_30'] = amount_cz['Amount'] / amount_cz['med_cz_30']



## Cardnum + state Amount Expert Variables

In [75]:
%%time
avg_cs_1 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('1d').mean().reset_index()
avg_cs_1['order_cs'] = avg_cs_1.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

avg_cs_7 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('7d').mean().reset_index()
avg_cs_7['order_cs'] = avg_cs_7.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

avg_cs_14 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('14d').mean().reset_index()
avg_cs_14['order_cs'] = avg_cs_14.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

avg_cs_30 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('30d').mean().reset_index()
avg_cs_30['order_cs'] = avg_cs_30.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1


tot_cs_1 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('1d').sum().reset_index()
tot_cs_1['order_cs'] = tot_cs_1.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

tot_cs_7 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('7d').sum().reset_index()
tot_cs_7['order_cs'] = tot_cs_7.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

tot_cs_14 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('14d').sum().reset_index()
tot_cs_14['order_cs'] = tot_cs_14.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

tot_cs_30 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('30d').sum().reset_index()
tot_cs_30['order_cs'] = tot_cs_30.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1


med_cs_1 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('1d').median().reset_index()
med_cs_1['order_cs'] = med_cs_1.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

med_cs_7 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('7d').median().reset_index()
med_cs_7['order_cs'] = med_cs_7.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

med_cs_14 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('14d').median().reset_index()
med_cs_14['order_cs'] = med_cs_14.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

med_cs_30 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('30d').median().reset_index()
med_cs_30['order_cs'] = med_cs_30.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1


max_cs_1 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('1d').max().reset_index()
max_cs_1['order_cs'] = max_cs_1.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

max_cs_7 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('7d').max().reset_index()
max_cs_7['order_cs'] = max_cs_7.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

max_cs_14 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('14d').max().reset_index()
max_cs_14['order_cs'] = max_cs_14.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

max_cs_30 = card_ref.groupby(['Cardnum', 'Merch state'])['Amount'].rolling('30d').max().reset_index()
max_cs_30['order_cs'] = max_cs_30.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1




CPU times: user 2min 13s, sys: 1.23 s, total: 2min 14s
Wall time: 2min 15s


In [76]:
amount_cs = card_clean.iloc[:, [0,1,2,5,8,14]].merge(avg_cs_1, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(avg_cs_7, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(avg_cs_14, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(avg_cs_30, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(tot_cs_1, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(tot_cs_7, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(tot_cs_14, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(tot_cs_30, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(max_cs_1, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(max_cs_7, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(max_cs_14, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(max_cs_30, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(med_cs_1, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(med_cs_7, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(med_cs_14, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner').merge(med_cs_30, on = ['Cardnum','Merch state', 'Date','order_cs'], how = 'inner')

amount_cs.columns = ['Recnum','Cardnum', 'Date', 'Merch state', 'Amount','order_cs', 'avg_cs_1', 'avg_cs_7', 'avg_cs_14',
                     'avg_cs_30', 'tot_cs_1', 'tot_cs_7', 'tot_cs_14', 'tot_cs_30', 'max_cs_1', 'max_cs_7', 'max_cs_14', 'max_cs_30',
                     'med_cs_1','med_cs_7','med_cs_14','med_cs_30' ]


In [77]:
## actual - average

amount_cs['daa_cs_1'] = amount_cs['Amount'] - amount_cs['avg_cs_1']

amount_cs['daa_cs_7'] = amount_cs['Amount'] - amount_cs['avg_cs_7']

amount_cs['daa_cs_14'] = amount_cs['Amount'] - amount_cs['avg_cs_14']

amount_cs['daa_cs_30'] = amount_cs['Amount'] - amount_cs['avg_cs_30']


## actual - median

amount_cs['dam_cs_1'] = amount_cs['Amount'] - amount_cs['med_cs_1']

amount_cs['dam_cs_7'] = amount_cs['Amount'] - amount_cs['med_cs_7']

amount_cs['dam_cs_14'] = amount_cs['Amount'] - amount_cs['med_cs_14']

amount_cs['dam_cs_30'] = amount_cs['Amount'] - amount_cs['med_cs_30']


## actual / average

amount_cs['qaa_cs_1'] = amount_cs['Amount'] / amount_cs['avg_cs_1']

amount_cs['qaa_cs_7'] = amount_cs['Amount'] / amount_cs['avg_cs_7']

amount_cs['qaa_cs_14'] = amount_cs['Amount'] / amount_cs['avg_cs_14']

amount_cs['qaa_cs_30'] = amount_cs['Amount'] / amount_cs['avg_cs_30']



## actual / max

amount_cs['qam_cs_1'] = amount_cs['Amount'] / amount_cs['max_cs_1']

amount_cs['qam_cs_7'] = amount_cs['Amount'] / amount_cs['max_cs_7']

amount_cs['qam_cs_14'] = amount_cs['Amount'] / amount_cs['max_cs_14']

amount_cs['qam_cs_30'] = amount_cs['Amount'] / amount_cs['max_cs_30']



## actual / total

amount_cs['qat_cs_1'] = amount_cs['Amount'] / amount_cs['tot_cs_1']

amount_cs['qat_cs_7'] = amount_cs['Amount'] / amount_cs['tot_cs_7']

amount_cs['qat_cs_14'] = amount_cs['Amount'] / amount_cs['tot_cs_14']

amount_cs['qat_cs_30'] = amount_cs['Amount'] / amount_cs['tot_cs_30']


## actual / median

amount_cs['qamed_cs_1'] = amount_cs['Amount'] / amount_cs['med_cs_1']

amount_cs['qamed_cs_7'] = amount_cs['Amount'] / amount_cs['med_cs_7']

amount_cs['qamed_cs_14'] = amount_cs['Amount'] / amount_cs['med_cs_14']

amount_cs['qamed_cs_30'] = amount_cs['Amount'] / amount_cs['med_cs_30']





## Frequency Expert Variables

In [78]:
%%time
## Cardnum

ntr_cardnum_1 = card_ref.groupby(['Cardnum'])['Recnum'].rolling('1d').count().reset_index()
ntr_cardnum_1['order'] = ntr_cardnum_1.groupby(['Cardnum', 'Date']).cumcount() + 1

ntr_cardnum_7 = card_ref.groupby(['Cardnum'])['Recnum'].rolling('7d').count().reset_index()
ntr_cardnum_7['order'] = ntr_cardnum_7.groupby(['Cardnum', 'Date']).cumcount() + 1

ntr_cardnum_14 = card_ref.groupby(['Cardnum'])['Recnum'].rolling('14d').count().reset_index()
ntr_cardnum_14['order'] = ntr_cardnum_14.groupby(['Cardnum', 'Date']).cumcount() + 1

ntr_cardnum_30 = card_ref.groupby(['Cardnum'])['Recnum'].rolling('30d').count().reset_index()
ntr_cardnum_30['order'] = ntr_cardnum_30.groupby(['Cardnum', 'Date']).cumcount() + 1



CPU times: user 4.35 s, sys: 72.1 ms, total: 4.42 s
Wall time: 4.5 s


In [79]:
%%time
## Merchnum

ntr_merch_1 = card_ref.groupby(['Merchnum'])['Recnum'].rolling('1d').count().reset_index()
ntr_merch_1['order_merch'] = ntr_merch_1.groupby(['Merchnum', 'Date']).cumcount() + 1

ntr_merch_7 = card_ref.groupby(['Merchnum'])['Recnum'].rolling('7d').count().reset_index()
ntr_merch_7['order_merch'] = ntr_merch_7.groupby(['Merchnum', 'Date']).cumcount() + 1

ntr_merch_14 = card_ref.groupby(['Merchnum'])['Recnum'].rolling('14d').count().reset_index()
ntr_merch_14['order_merch'] = ntr_merch_14.groupby(['Merchnum', 'Date']).cumcount() + 1

ntr_merch_30 = card_ref.groupby(['Merchnum'])['Recnum'].rolling('30d').count().reset_index()
ntr_merch_30['order_merch'] = ntr_merch_30.groupby(['Merchnum', 'Date']).cumcount() + 1

CPU times: user 35.2 s, sys: 481 ms, total: 35.6 s
Wall time: 37.2 s


In [80]:
%%time
## cardnum + merch
ntr_cm_1 = card_ref.groupby(['Cardnum','Merchnum'])['Recnum'].rolling('1d').count().reset_index()
ntr_cm_1['order_cm'] = ntr_cm_1.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

ntr_cm_7 = card_ref.groupby(['Cardnum','Merchnum'])['Recnum'].rolling('7d').count().reset_index()
ntr_cm_7['order_cm'] = ntr_cm_7.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

ntr_cm_14 = card_ref.groupby(['Cardnum','Merchnum'])['Recnum'].rolling('14d').count().reset_index()
ntr_cm_14['order_cm'] = ntr_cm_14.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1

ntr_cm_30 = card_ref.groupby(['Cardnum','Merchnum'])['Recnum'].rolling('30d').count().reset_index()
ntr_cm_30['order_cm'] = ntr_cm_30.groupby(['Cardnum','Merchnum', 'Date']).cumcount() + 1



CPU times: user 1min 25s, sys: 1.04 s, total: 1min 26s
Wall time: 1min 28s


In [81]:
%%time
## cardnum + zip

ntr_cz_1 = card_ref.groupby(['Cardnum','Merch zip'])['Recnum'].rolling('1d').count().reset_index()
ntr_cz_1['order_cz'] = ntr_cz_1.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

ntr_cz_7 = card_ref.groupby(['Cardnum','Merch zip'])['Recnum'].rolling('7d').count().reset_index()
ntr_cz_7['order_cz'] = ntr_cz_7.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

ntr_cz_14 = card_ref.groupby(['Cardnum','Merch zip'])['Recnum'].rolling('14d').count().reset_index()
ntr_cz_14['order_cz'] = ntr_cz_14.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1

ntr_cz_30 = card_ref.groupby(['Cardnum','Merch zip'])['Recnum'].rolling('30d').count().reset_index()
ntr_cz_30['order_cz'] = ntr_cz_30.groupby(['Cardnum','Merch zip', 'Date']).cumcount() + 1



## cardnum + state

ntr_cs_1 = card_ref.groupby(['Cardnum','Merch state'])['Recnum'].rolling('1d').count().reset_index()
ntr_cs_1['order_cs'] = ntr_cs_1.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

ntr_cs_7 = card_ref.groupby(['Cardnum','Merch state'])['Recnum'].rolling('7d').count().reset_index()
ntr_cs_7['order_cs'] = ntr_cs_7.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

ntr_cs_14 = card_ref.groupby(['Cardnum','Merch state'])['Recnum'].rolling('14d').count().reset_index()
ntr_cs_14['order_cs'] = ntr_cs_14.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1

ntr_cs_30 = card_ref.groupby(['Cardnum','Merch state'])['Recnum'].rolling('30d').count().reset_index()
ntr_cs_30['order_cs'] = ntr_cs_30.groupby(['Cardnum','Merch state', 'Date']).cumcount() + 1



CPU times: user 1min 42s, sys: 976 ms, total: 1min 43s
Wall time: 1min 44s


## Day Since Expert Variables

In [82]:
%%time
## Cardnum

card_ref = card_ref.reset_index()

day_card = card_ref.groupby(['Cardnum', 'Date'])['Cardnum', 'Date'].first().groupby('Cardnum').diff()

day_card.columns = ['card_diff']

day_card = day_card.reset_index()

## merchnum

day_merch = card_ref.groupby(['Merchnum', 'Date'])['Merchnum', 'Date'].first().groupby('Merchnum').diff()

day_merch.columns = ['merch_diff']

day_merch = day_merch.reset_index()

## cardnum + merch

day_cm = card_ref.groupby(['Cardnum','Merchnum', 'Date'])['Cardnum', 'Merchnum', 'Date'].first().groupby(['Cardnum', 'Merchnum']).diff()

day_cm.columns = ['cm_diff']

day_cm = day_cm.reset_index()


## cardnum + zip

day_cz = card_ref.groupby(['Cardnum','Merch zip', 'Date'])['Cardnum', 'Merch zip', 'Date'].first().groupby(['Cardnum', 'Merch zip']).diff()

day_cz.columns = ['cz_diff']

day_cz = day_cz.reset_index()

## cardnum + state

day_cs = card_ref.groupby(['Cardnum','Merch state', 'Date'])['Cardnum', 'Merch state', 'Date'].first().groupby(['Cardnum', 'Merch state']).diff()

day_cs.columns = ['cs_diff']

day_cs = day_cs.reset_index()

/Users/stevecoggeshall/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: 'Cardnum' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/Users/stevecoggeshall/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: 'Merchnum' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/Users/stevecoggeshall/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: FutureWarning: 'Cardnum' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/Users/stevecoggeshall/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: FutureWarning: 'Merchnum' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/Users/stevecoggeshall/anaconda3/lib/python3.5/site-pac

CPU times: user 1min 27s, sys: 1.51 s, total: 1min 29s
Wall time: 1min 29s


## Velocity Expert Variables

In [83]:
%%time
## cardnum

freq_card = ntr_cardnum_1.merge(ntr_cardnum_7, how = 'inner', on = ['Cardnum', 'Date', 'order']).merge(ntr_cardnum_14, how = 'inner', on = ['Cardnum', 'Date', 'order']).merge(ntr_cardnum_30, how = 'inner', on = ['Cardnum', 'Date', 'order'])

freq_card.columns = ['Cardnum', 'Date', 'ntr_cardnum_1', 'order', 'ntr_cardnum_7', 'ntr_cardnum_14', 'ntr_cardnum_30']

freq_card['v_card_7'] = freq_card['ntr_cardnum_1'] / freq_card['ntr_cardnum_7'] / 7

freq_card['v_card_14'] = freq_card['ntr_cardnum_1'] / freq_card['ntr_cardnum_14'] / 14

freq_card['v_card_30'] = freq_card['ntr_cardnum_1'] / freq_card['ntr_cardnum_30'] / 30



CPU times: user 161 ms, sys: 38.2 ms, total: 199 ms
Wall time: 198 ms


In [84]:
%%time
## merchant

freq_merch = ntr_merch_1.merge(ntr_merch_7, how = 'inner', on = ['Merchnum', 'Date', 'order_merch']).merge(ntr_merch_14, how = 'inner', on = ['Merchnum', 'Date', 'order_merch']).merge(ntr_merch_30, how = 'inner', on = ['Merchnum', 'Date', 'order_merch'])

freq_merch.columns = ['Merchnum', 'Date', 'ntr_merch_1', 'order_merch', 'ntr_merch_7', 'ntr_merch_14', 'ntr_merch_30']

freq_merch['v_merch_7'] = freq_merch['ntr_merch_1'] / freq_merch['ntr_merch_7'] / 7

freq_merch['v_merch_14'] = freq_merch['ntr_merch_1'] / freq_merch['ntr_merch_14'] / 14

freq_merch['v_merch_30'] = freq_merch['ntr_merch_1'] / freq_merch['ntr_merch_30'] / 30

CPU times: user 191 ms, sys: 21.1 ms, total: 213 ms
Wall time: 211 ms


In [85]:
freq_merch.shape

(96397, 10)

## Compile Variables for Cardnum

In [86]:
card_var = amount_card.merge(freq_card, how = 'inner', on = ['Cardnum', 'Date', 'order']).merge(day_card, how = 'inner', on = ['Cardnum', 'Date'])

In [87]:
card_var = card_var.drop(columns = ['Cardnum', 'Date', 'Amount', 'order'])

In [88]:
card_var.head()

,Recnum,avg_card_1,avg_card_7,avg_card_14,avg_card_30,tot_card_1,tot_card_7,tot_card_14,tot_card_30,max_card_1,...,qamed_card_14,qamed_card_30,ntr_cardnum_1,ntr_cardnum_7,ntr_cardnum_14,ntr_cardnum_30,v_card_7,v_card_14,v_card_30,card_diff
0,1,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.0,1.0,1.0,1.0,1.0,1.0,0.142857,0.071429,0.033333,NaT
1,5,3.62,3.62,3.62,3.62,7.24,7.24,7.24,7.24,3.62,...,1.0,1.0,2.0,2.0,2.0,2.0,0.142857,0.071429,0.033333,NaT
2,10,3.62,3.62,3.62,3.62,10.86,10.86,10.86,10.86,3.62,...,1.0,1.0,3.0,3.0,3.0,3.0,0.142857,0.071429,0.033333,NaT
3,12,3.62,3.62,3.62,3.62,14.48,14.48,14.48,14.48,3.62,...,1.0,1.0,4.0,4.0,4.0,4.0,0.142857,0.071429,0.033333,NaT
4,21,3.62,3.62,3.62,3.62,18.10,18.10,18.10,18.10,3.62,...,1.0,1.0,5.0,5.0,5.0,5.0,0.142857,0.071429,0.033333,NaT


## Complile Variables for Merch

In [89]:
merch_var = amount_merch.merge(freq_merch, how = 'inner', on = ['Merchnum', 'Date', 'order_merch']).merge(day_merch, how = 'inner', on = ['Merchnum', 'Date'])


In [90]:
merch_var = merch_var.drop(columns = ['Merchnum', 'Date', 'Amount', 'order_merch'])

In [91]:
merch_var.head()

,Recnum,avg_merch_1,avg_merch_7,avg_merch_14,avg_merch_30,tot_merch_1,tot_merch_7,tot_merch_14,tot_merch_30,med_merch_1,...,qamed_merch_14,qamed_merch_30,ntr_merch_1,ntr_merch_7,ntr_merch_14,ntr_merch_30,v_merch_7,v_merch_14,v_merch_30,merch_diff
0,1,3.6200,3.6200,3.6200,3.6200,3.62,3.62,3.62,3.62,3.62,...,1.000000,1.000000,1.0,1.0,1.0,1.0,0.142857,0.071429,0.033333,NaT
1,4,3.6200,3.6200,3.6200,3.6200,7.24,7.24,7.24,7.24,3.62,...,1.000000,1.000000,2.0,2.0,2.0,2.0,0.142857,0.071429,0.033333,NaT
2,5,3.6200,3.6200,3.6200,3.6200,10.86,10.86,10.86,10.86,3.62,...,1.000000,1.000000,3.0,3.0,3.0,3.0,0.142857,0.071429,0.033333,NaT
3,6,3.6325,3.6325,3.6325,3.6325,14.53,14.53,14.53,14.53,3.62,...,1.013812,1.013812,4.0,4.0,4.0,4.0,0.142857,0.071429,0.033333,NaT
4,7,3.6300,3.6300,3.6300,3.6300,18.15,18.15,18.15,18.15,3.62,...,1.000000,1.000000,5.0,5.0,5.0,5.0,0.142857,0.071429,0.033333,NaT


## Compile Variables for Cardnum + Merch

In [92]:
## combine frequency

freq_cm = ntr_cm_1.merge(ntr_cm_7, how = 'inner', on = ['Cardnum', 'Merchnum', 'Date', 'order_cm']).merge(ntr_cm_14, how = 'inner', on = ['Cardnum', 'Merchnum', 'Date', 'order_cm']).merge(ntr_cm_30, how = 'inner', on = ['Cardnum', 'Merchnum', 'Date', 'order_cm'])

freq_cm.columns = ['Cardnum', 'Merchnum', 'Date', 'ntr_cm_1', 'order_cm', 'ntr_cm_7', 'ntr_cm_14', 'ntr_cm_30']



freq_cz = ntr_cz_1.merge(ntr_cz_7, how = 'inner', on = ['Cardnum', 'Merch zip', 'Date', 'order_cz']).merge(ntr_cz_14, how = 'inner', on = ['Cardnum', 'Merch zip', 'Date', 'order_cz']).merge(ntr_cz_30, how = 'inner', on = ['Cardnum', 'Merch zip', 'Date', 'order_cz'])

freq_cz.columns = ['Cardnum', 'Merch zip', 'Date', 'ntr_cz_1', 'order_cz', 'ntr_cz_7', 'ntr_cz_14', 'ntr_cz_30']


freq_cs = ntr_cs_1.merge(ntr_cs_7, how = 'inner', on = ['Cardnum', 'Merch state', 'Date', 'order_cs']).merge(ntr_cs_14, how = 'inner', on = ['Cardnum', 'Merch state', 'Date', 'order_cs']).merge(ntr_cs_30, how = 'inner', on = ['Cardnum', 'Merch state', 'Date', 'order_cs'])

freq_cs.columns = ['Cardnum', 'Merch state', 'Date', 'ntr_cs_1', 'order_cs', 'ntr_cs_7', 'ntr_cs_14', 'ntr_cs_30']



In [93]:
cm_var = amount_cm.merge(freq_cm, how = 'inner', on = ['Cardnum', 'Merchnum', 'Date', 'order_cm']).merge(day_cm, how = 'inner', on = ['Cardnum', 'Merchnum', 'Date'])


In [94]:
cm_var = cm_var.drop(columns = ['Cardnum', 'Date', 'Merchnum', 'Amount', 'order_cm'])

In [95]:
cm_var.head()

,Recnum,avg_cm_1,avg_cm_7,avg_cm_14,avg_cm_30,tot_cm_1,tot_cm_7,tot_cm_14,tot_cm_30,max_cm_1,...,qat_cm_30,qamed_cm_1,qamed_cm_7,qamed_cm_14,qamed_cm_30,ntr_cm_1,ntr_cm_7,ntr_cm_14,ntr_cm_30,cm_diff
0,1,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaT
1,5,3.62,3.62,3.62,3.62,7.24,7.24,7.24,7.24,3.62,...,0.500000,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaT
2,10,3.62,3.62,3.62,3.62,10.86,10.86,10.86,10.86,3.62,...,0.333333,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,NaT
3,12,3.62,3.62,3.62,3.62,14.48,14.48,14.48,14.48,3.62,...,0.250000,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,NaT
4,21,3.62,3.62,3.62,3.62,18.10,18.10,18.10,18.10,3.62,...,0.200000,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,NaT


## Compile Variables for Cardnum + zip

In [96]:
cz_var = amount_cz.merge(freq_cz, how = 'inner', on = ['Cardnum', 'Merch zip', 'Date', 'order_cz']).merge(day_cz, how = 'inner', on = ['Cardnum', 'Merch zip', 'Date'])



In [97]:
cz_var = cz_var.drop(columns = ['Cardnum', 'Date', 'Merch zip', 'Amount', 'order_cz'])

In [98]:
cz_var.head()

,Recnum,avg_cz_1,avg_cz_7,avg_cz_14,avg_cz_30,tot_cz_1,tot_cz_7,tot_cz_14,tot_cz_30,max_cz_1,...,qat_cz_30,qamed_cz_1,qamed_cz_7,qamed_cz_14,qamed_cz_30,ntr_cz_1,ntr_cz_7,ntr_cz_14,ntr_cz_30,cz_diff
0,1,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaT
1,5,3.62,3.62,3.62,3.62,7.24,7.24,7.24,7.24,3.62,...,0.500000,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaT
2,10,3.62,3.62,3.62,3.62,10.86,10.86,10.86,10.86,3.62,...,0.333333,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,NaT
3,12,3.62,3.62,3.62,3.62,14.48,14.48,14.48,14.48,3.62,...,0.250000,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,NaT
4,21,3.62,3.62,3.62,3.62,18.10,18.10,18.10,18.10,3.62,...,0.200000,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,NaT


## Compile Variables for Cardnum + state

In [99]:
cs_var = amount_cs.merge(freq_cs, how = 'inner', on = ['Cardnum', 'Merch state', 'Date', 'order_cs']).merge(day_cs, how = 'inner', on = ['Cardnum', 'Merch state', 'Date'])



In [100]:
cs_var = cs_var.drop(columns = ['Cardnum', 'Date', 'Merch state', 'Amount', 'order_cs'])

In [101]:
cs_var.head()

,Recnum,avg_cs_1,avg_cs_7,avg_cs_14,avg_cs_30,tot_cs_1,tot_cs_7,tot_cs_14,tot_cs_30,max_cs_1,...,qat_cs_30,qamed_cs_1,qamed_cs_7,qamed_cs_14,qamed_cs_30,ntr_cs_1,ntr_cs_7,ntr_cs_14,ntr_cs_30,cs_diff
0,1,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaT
1,5,3.62,3.62,3.62,3.62,7.24,7.24,7.24,7.24,3.62,...,0.500000,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaT
2,10,3.62,3.62,3.62,3.62,10.86,10.86,10.86,10.86,3.62,...,0.333333,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,NaT
3,12,3.62,3.62,3.62,3.62,14.48,14.48,14.48,14.48,3.62,...,0.250000,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,NaT
4,21,3.62,3.62,3.62,3.62,18.10,18.10,18.10,18.10,3.62,...,0.200000,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,NaT


## Compile All Variables

In [102]:
all_var = card_clean.loc[:,'Recnum':'Fraud'].merge(card_var, how = 'inner', on = 'Recnum').merge(merch_var, how = 'inner', on = 'Recnum').merge(cm_var, how = 'inner', on = 'Recnum').merge(cz_var, how = 'inner', on = 'Recnum').merge(cs_var, how = 'inner', on = 'Recnum')



In [103]:
pd.options.display.max_rows = 250

In [104]:
all_var.isnull().sum()

Recnum                   0
Cardnum                  0
Date                     0
Merchnum                 0
Merch description        0
Merch state              0
Merch zip                0
Transtype                0
Amount                   0
Fraud                    0
avg_card_1               0
avg_card_7               0
avg_card_14              0
avg_card_30              0
tot_card_1               0
tot_card_7               0
tot_card_14              0
tot_card_30              0
max_card_1               0
max_card_7               0
max_card_14              0
max_card_30              0
med_card_1               0
med_card_7               0
med_card_14              0
med_card_30              0
daa_card_1               0
daa_card_7               0
daa_card_14              0
daa_card_30              0
dam_card_1               0
dam_card_7               0
dam_card_14              0
dam_card_30              0
qaa_card_1               0
qaa_card_7               0
qaa_card_14              0
q

In [105]:
all_var_final = all_var.fillna(0)

In [106]:
all_var_final.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,qat_cs_30,qamed_cs_1,qamed_cs_7,qamed_cs_14,qamed_cs_30,ntr_cs_1,ntr_cs_7,ntr_cs_14,ntr_cs_30,cs_diff
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0 days
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0 days
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0 days
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0 days
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,0.5,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,0 days


In [107]:
all_var_final.to_csv('all_vars', index = False)

In [108]:
all_var_final.shape

(96397, 241)